This notebook contains the first level MRI analyses: setting up and examining a General Linear Model that produces response amplitudes for each stimulus class. For this, the data must be preprocessed. We will use GLMdenoise to do this, which is a MATLAB toolbox, and so the actual analysis will be run outside of this notebook (it takes a while and a lot of resources anyway, so this is for the best).

# Creating the design matrix

The first thing we need to do is create our design matrix. Our design matrix needs to be in the format time by conditions (where time is in TRs), with a 1 for condition onset. This will be exceedingly sparse, since each condition only shows up once per run (when we move this into matlab, we will make it a sparse matrix). We will have one of these design matrices per run.

In [ ]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
results = h5py.File('../data/raw_behavioral/2017-Nov-07_wl_subj042_sess0.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

Backticks from the scanner are recorded as 5s, which we hold onto, allowing us to quickly see how many TRs we recorded in each run (for some sessions, the last run is empty because we quit it as soon as it started; this happened because we ran out of time and had to end the scanning session earlier and so did not gather the 12 runs we were hoping for):

In [ ]:
for run in range(12):
    if 'run_%02d_button_presses' % run in results:
        n_TRs = sum(['5' in i[0] for i in results['run_%02d_button_presses' % run].value])
        print("Run %s: %s TRs" % (run, n_TRs))


We also have the onset times of all of our stimuli, as well as an identifying index we can use to look up its information. We know that we presented stimuli in classes, each of which contains 8 stimuli, but we can reconstruct that information by using the indices to look into the dataframe we loaded in.

Based on how we constructed our stimuli, two stimuli belong to the same class iff they have the same `w_a` and `w_r` (angular and radial frequency) values.

In [ ]:
reduced_df = df[['w_r', 'w_a', 'index']].set_index('index')
reduced_df = reduced_df.reindex(results['run_01_shuffled_indices'].value)

As can be seen by examining this reordered dataframe, our classes occur in batches of 8 (note that there are some that have `NaN`s for both values; these are the blank stimuli).

In [ ]:
reduced_df

One way to discover the batch size programmatically is to find the smallest amount we have to jump by such that each subsequent entry has a different `w_r` and a different `w_a`.

In [ ]:
class_size = 0
break_out = False
while ~break_out:
    class_size += 1
    w_r = reduced_df['w_r'].values.copy()
    w_a = reduced_df['w_a'].values.copy()
    # we replace the NaNs with zeros for this calculation -- we want them be different than all
    # the other classes (and technically, the blank stimuli do have 0s in both w_r and w_a)
    nan_replace = 0
    w_r[np.isnan(w_r)] = nan_replace
    w_a[np.isnan(w_a)] = nan_replace
    tmp = np.abs(w_r[:-class_size:class_size] - w_r[class_size::class_size]) + np.abs(w_a[:-class_size:class_size] - w_a[class_size::class_size])
    class_changes = np.nonzero(tmp)[0]
    indices = np.array(range(len(tmp)))
    if len(class_changes) == len(indices):
        break_out = np.equal(class_changes, indices).all()
print("Each class is of size %s" % class_size)

We want to know what the class index of each stimulus is then, which we can find by dividing the index by the number of stimuli in each class:

In [ ]:
reduced_df['class_idx'] = reduced_df.index/class_size

Now we want to figure out what TR each stimulus was presented during. We have the time (in seconds) they appeared on screen recorded, so we want to add that information to our dataframe. In order to do so, we will first subtract off the time of the first TR (so we now have the "time of presentation from first TR") and throw out a bit of extra information. We also recorded when the start screen was turned off (that's the first entry) and there are a final number of blank stimuli, which we'll throw away as well

In [ ]:
timing = results['run_01_timing_data'].value
# Because we want to skip the first one and drop the last nblanks * 2 (since for each stimuli
# we have two entries: one for on, one for off). Finally, we only grab every other because we
# only want the on timing
timing = timing[1:-results['run_01_nblanks'].value*2:2]

# Now we get rid of the first TR
initial_TR_time = float(results['run_01_button_presses'].value[0][1])
timing = [float(i[2]) - initial_TR_time for i in timing]

# and add to our dataframe
reduced_df['Onset time (sec)'] = timing

Now we simply look for where there's a class transition, which happens every `class_size` (8) stimuli

In [ ]:
design_df = reduced_df[::class_size]

Now we need to convert these to to TR times. First we find the onset times of TRs, in seconds, relative to the first TR. We then create a giant matrix where each row is a different stimulus onset time and then, in each column, subtract a TR onset time (so this matrix will be `num_conditions x num_TRs`). If we then round this difference-in-time matrix and look for the 0s, we've found what TR onset is closest to the onset of the stimuli. Note that this won't make sense for a lot of entries; some of them start almost exactly halfway through a TR. But, because of how we defined our experiment, our class transitions should happen right around a TR onset (if the timings of the scanner and the stimulus computer were perfect, then they would be exactly the same; as it is they probably differ by a few milliseconds) and so this will work

In [ ]:
# 5 indicates a backtick from the scanner
TR_times = np.array([float(i[1]) for i in results['run_01_button_presses'].value if '5'==i[0]])
TR_times -= TR_times[0]

In [ ]:
stim_times = design_df['Onset time (sec)'].values
stim_times = np.expand_dims(stim_times,1)
stim_times = np.repeat(stim_times,len(TR_times),1)

In [ ]:
time_from_TR = np.round(stim_times - TR_times)
design_df['Onset time (TR)'] = np.where(time_from_TR==0)[1]

And we create our design matrix, iterate through throw our `design_df` and put a one where each class shows up in a TR.

In [ ]:
# Our blanks show up as having nan values, and we don't want to model them in our GLM, so we drop them.
design_df = design_df.dropna()
# because the values are 0-indexed
design_matrix = np.zeros((len(TR_times), design_df.class_idx.max()+1))

for i, row in design_df.iterrows():
    row = row.astype(int)
    design_matrix[row['Onset time (TR)'], row['class_idx']] = 1

To make sure things work correctly, we look at our axis sums: each class (axis 1) should show up exactly once and each TR (axis 0) should have 0 or 1 classes in it

In [ ]:
print("Each entry represents one of our %d classes:" % design_matrix.shape[1])
print(design_matrix.sum(0))

print("Each entry represents one of our %d TRs:" % design_matrix.shape[0])
print(design_matrix.sum(1))


And now we can look at our design matrix for this run!

In [ ]:
ax = plt.imshow(design_matrix, 'gray', aspect='auto',)
plt.title("Design matrix for run 1")
plt.xlabel("events")
plt.ylabel("TRs")

The function `sfp.first_level_analysis.create_design_matrix` does the above (without the visualizations and checks) and returns the resulting design matrix. The function `sfp.first_level_analysis.create_all_design_matrices` does this for multiple runs and saves them as `.mat` files so they can be read into matlab.

Actually running the first-level analysis requires matlab and should be run on the cluster (see `matlab/runGLM.m` and `matlab/runGLM.sbatch`), since they require Kendrick Kay's [GLMdenoise](http://kendrickkay.net/GLMdenoise/) package and use a lot of memory. After you've finished getting the results, examined the $R^2$ values to make sure they make sense, and realigned them to the subject's freesurfer anatomy (using `sfp.realign`), then you're ready for the following section, where we analyze these results.

# Analyzing the first-level results

After running our GLM analysis, we have the estimated amplitude responses of each voxel to each image class. Along with Noah Benson's anatomical template / Bayesian model, we also have each voxel's visual area and location in the visual field (in terms of eccentricity and polar angle). By combining the information contained within them, along with the dataframe describing each stimulus class, we can construct our tuning curves.

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import seaborn as sns
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import itertools
import pyPyrTools.JBhelpers as jbh
import pyPyrTools as ppt
import scipy as sp

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
behav_results = h5py.File('../data/raw_behavioral/2017-Nov-07_wl_subj042_sess0.hdf5')
#behav_results = h5py.File('../data/raw_behavioral/2017-Nov-07_wl_subj045_sess0.hdf5')
#behav_results = h5py.File('../data/raw_behavioral/2017-Oct-09_wl_subj001_sess1.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
stim_df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

# Array full of the actual stimuli
stim = np.load('../data/stimuli/unshuffled.npy')

# for this, we just want any run, since they all contain the same classes and we don't care about their order
design_df, _, _ = sfp.first_level_analysis.create_design_df(behav_results, stim_df, 1)
design_df = design_df.reset_index(drop=True).sort_values(by="class_idx")
design_df = design_df[['w_r', 'w_a', 'class_idx']].set_index('class_idx')

stim_df = stim_df.set_index(['w_r', 'w_a'])
stim_df['class_idx'] = design_df.reset_index().set_index(['w_r', 'w_a'])['class_idx']
stim_df = stim_df.reset_index()

In [ ]:
df = pd.read_csv('/home/billbrod/Desktop/wl_subj045_bootstrapped.csv')

This doesn't work right now, it's averaging across the different visual areas

Also, sometimes it's "varea" in benson templates, sometimes it's "areas"

In [ ]:
subject_name = 'wl_subj001'
benson_path = "%s/%s/surf/{}.benson14_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_name)
results_path = "/mnt/Acadia/Projects/spatial_frequency_preferences/%s/20171007_prisma/MRI_first_level/results/stim_class/{}-{}.mgz" % subject_name
benson_path = benson_path.replace('{}', '%s')
results_path = results_path.replace('{}', '%s')

df = sfp.first_level_analysis.create_GLM_result_df(design_df, benson_path, results_path, 'full', '/home/billbrod/Desktop/wl_subj001_bootstrapped.csv', vareas=[1])

In [ ]:
subject_name = 'wl_subj042'
benson_path = "%s/%s/surf/{}.benson14_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_name)
results_path = "/mnt/Acadia/Projects/spatial_frequency_preferences/%s/20171107/MRI_first_level/stim_class/results/{}-{}.mgz" % subject_name
benson_path = benson_path.replace('{}', '%s')
results_path = results_path.replace('{}', '%s')

df = sfp.first_level_analysis.create_GLM_result_df(design_df, benson_path, results_path, 'full', '~/Desktop/wl_subj042_bootstrapped.csv', vareas=[1])

In [ ]:
subject_name = 'wl_subj045'
# benson_path = "%s/%s/surf/{}.benson14_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_name)
benson_path = '/home/billbrod/Desktop/wl_subj045/surf/%s.template_%s.mgz'
results_path = "/mnt/Acadia/Projects/spatial_frequency_preferences/%s/20171107/MRI_first_level/stim_class/results/{}-{}.mgz" % subject_name
# benson_path = benson_path.replace('{}', '%s')
results_path = results_path.replace('{}', '%s')

df = sfp.first_level_analysis.create_GLM_result_df(design_df, benson_path, results_path, 'full', '~/Desktop/wl_subj045_bootstrapped.csv', vareas=[1])

Here we see the different stimulus classes, as plotted in frequency space, colored by their superclass. These numbers are roughly log-spaced (doubling).

In [ ]:
with sns.axes_style('whitegrid'):
    g = sns.FacetGrid(df[df.voxel==0], hue='stimulus_superclass', size=5, aspect=1)
    g.map(sns.plt.scatter, 'w_a', 'w_r')
    g.add_legend()
    _=g.ax.axis('equal')

We also want to be able to see the amplitude response as a function of local spatial frequency. For now, I only do this for `w_r` alone (so, circular stimuli), because there are some details to work out: how to make the `w_a` spatial frequency map respect the `alpha` value and how to combine the two maps in order to get appropriate values for the spirals and mixtures

In [ ]:
def calculate_stim_cpd(w_r, w_a=0, max_degree_rad=12, alpha=50, stim_size=1080, plot_flag=False):
    _, w_r = sfp.stimuli.create_sf_maps_cpd(stim_size, alpha, max_degree_rad*2, w_r, w_a)
    R = ppt.mkR(stim_size)

    # if max_degree_rad corresponds to the max vertical/horizontal extent, the actual max will be
    # np.sqrt(2*max_degree_rad**2) (this corresponds to the far corner)
    # this should be the radius of the screen, because R starts from the center and goes to the edge 
    R = R/(R.max()/np.sqrt(2*max_degree_rad**2))

    # jbh.showIm([w_r, R], ncols=2)

    # restrict to the eccentricity range we're looking at
    bin_masks = []
    eccen_idx = []
    for i in range(2, 8):
        bin_masks.append((R>i)&(R<(i+1)))
        eccen_idx.append('%s-%s' % (i, i+1))

    eccen_local_freqs = []
    eccens = []
    for m in bin_masks:
        eccens.append(R[m].mean())
        eccen_local_freqs.append(w_r[m].mean())

    if plot_flag:
        sns.plt.plot(eccens, eccen_local_freqs)
        ax = sns.plt.gca()
        ax.set_title('Spatial frequency vs eccentricity')
        ax.set_xlabel('Eccentricity (degrees)')
        _=ax.set_ylabel('Spatial frequency (cpd)')

    return pd.Series(eccen_local_freqs, eccen_idx)

_ = calculate_stim_cpd(181, plot_flag=True)

In [ ]:
# these are the stimuli we want to calculate the above curves for
w_r = df[df.stimulus_superclass=='circular'].w_r.unique()
sfs = []

for r in w_r:
    t = calculate_stim_cpd(r)
    t = pd.DataFrame(t, columns=['Local spatial frequency (cpd)'])
    t.index.name = 'eccen'
    t['w_r'] = r
    sfs.append(t)

sfs = pd.concat(sfs)

sfs['stimulus_superclass'] = 'circular'
sfs['w_a'] = 0
sfs = sfs.reset_index()

sfs = sfs.set_index(['stimulus_superclass', 'w_a', 'w_r', 'eccen'])
df = df.set_index(['stimulus_superclass', 'w_a', 'w_r', 'eccen'])
df = df.join(sfs)
df = df.reset_index()

In [ ]:
Rmin, Rmax = sfp.first_level_analysis.find_ecc_range_in_degrees(stim[0,:,:], 14)
print("Inside radius of stimulus annulus: %.02f" % Rmin)
print("Outside radius of stimulus annulus: %.02f" % Rmax)

In [ ]:
tmp_df = df[df.stimulus_superclass=='circular']
g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=5,)
#g.map(sns.regplot, 'rounded_freq_space_distance', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'rounded_freq_space_distance', 'amplitude_estimate')
g.map_dataframe(sfp.utils.scatter_ci_dist, 'freq_space_distance', 'amplitude_estimate')
for ax in g.axes.flatten():
    ax.set_xscale('log', basex=2)
g.add_legend()
g.fig.savefig('wl_subj042-raw.svg')

In [ ]:
g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=5, col='stimulus_superclass', col_wrap=2,
                  col_order=['circular', 'radial', 'forward spiral', 'reverse spiral', 'mixtures'])
#g.map(sns.regplot, 'freq_space_distance', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_distance', 'amplitude_estimate')
g.map_dataframe(sfp.utils.scatter_ci_dist, 'freq_space_distance', 'amplitude_estimate')
for ax in g.axes:
    ax.set_xscale('log', basex=2)
g.add_legend()

If there were no scaling in the visual system, such that neurons at different places in the visual field were expected to have similar properties, the bottom would all line up well, and it doesn't!

In [ ]:
classes_of_interest = []
classes_of_interest.extend(df[(df.stimulus_superclass=='circular')&(df.rounded_freq_space_distance==11)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='radial')&(df.rounded_freq_space_distance==64)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='forward spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='reverse spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
#classes_of_interest.append(df[df.stimulus_superclass=='mixtures'].stimulus_class.values[0])

stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

tmp_df = df[df.stimulus_superclass=='mixtures']
g = sns.FacetGrid(tmp_df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

In [ ]:
angles = sorted(df.freq_space_angle.unique())
stim_idxs = []
for ang in angles:
    class_of_interest = df[(df.freq_space_angle==ang)&(df.rounded_freq_space_distance==32)].stimulus_class.unique()[0]
    stim_idxs.append(stim_df[stim_df.class_idx==class_of_interest].index[0])

#stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
tmp_df = pd.DataFrame(df.groupby(['eccen_bin', 'w_r', 'w_a']).modelmd.mean()).reset_index()

g = sns.FacetGrid(tmp_df, col='eccen_bin', col_wrap=4, size=5)
cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes
g.map(sfp.utils.scatter_heat, 'w_a', 'w_r', 'amplitude_estimate_median', vmin=tmp_df['amplitude_estimate_median'].min(), 
      vmax=tmp_df['amplitude_estimate_median'].max())
sns.plt.colorbar(cax=cbar_ax)
g.fig.subplots_adjust(right=.9, top=.9)
g.fig.suptitle('Average response amplitude estimates at each point in frequency space')

In [ ]:
tmp_df = pd.pivot_table(df, 'amplitude_estimate_median', 'rounded_freq_space_distance', 'eccen_bin')
norm_df = tmp_df.copy()
for col in norm_df.columns:
    norm_df[col] = norm_df[col] / norm_df[col].max()

In [ ]:
fig = sns.heatmap(tmp_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Average response amplitude estimates')

In [ ]:
fig = sns.heatmap(norm_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Normalized average response amplitude estimates')

In [ ]:
sns.distplot(df.R2.values)

# Create plots for first year talk

In [ ]:
# for this, we're only using circular results
tmp_df = df[df.stimulus_superclass=='circular']
# tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num']]

In [ ]:
tmper_df = tmp_df[tmp_df.freq_space_distance==6.]
tmper_df = tmper_df.groupby(['eccen', 'bootstrap_num'])['amplitude_estimate'].mean().unstack().reset_index()

In [ ]:
hyp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num'], var_name='Frequency')
hyp_df = pd.DataFrame(hyp_df.groupby(['Frequency', 'value', 'bootstrap_num'])['amplitude_estimate'].mean()).reset_index()

In [ ]:
print("freq_space_distance min: %.03f, max: %.03f" % (tmp_df.freq_space_distance.min(), tmp_df.freq_space_distance.max()))
print("Halfway in log space: %.03f" % np.floor(2**((np.log2(181.) + np.log2(6.))/2.)))
# because these are circular, freq_space_distance==w_r
#stim_idx = stim_df[(stim_df.w_r.isin([6, 32, 181]))&(stim_df.w_a==0)].index[::8]
stim_idx = stim_df[(stim_df.w_r.isin([6, 32, ]))&(stim_df.w_a==0)].index[::8]
stims = [stim[i] for i in stim_idx]

We actually want to plot windows of the stimuli instead of just sins

In [ ]:
max_degree_rad = 12
scale_factor = 10
mask = sfp.utils.create_circle_mask(750, 350, scale_factor* 1080/(2*2*max_degree_rad), 1080)
stim_windows = [mask * s + ~mask.astype(bool)*127 for s in stims]


In [ ]:
import warnings
def fit_log_norm_ci(x, y, ci_vals=[2.5, 97.5], **kwargs):
    """fit log norm to data and plot the result

    to be used with seaborn.FacetGrid.map_dataframe
    """
    data = kwargs.pop('data')
    color = kwargs.pop('color')
    lines = []
    # we want to collapse hemispheres -- eventually this should be moved earlier
    tmp = pd.DataFrame(data.groupby([x, 'bootstrap_num'])[y].mean()).reset_index()
    for boot in data.bootstrap_num.unique():
        plot_data = tmp.groupby(x)[[y, 'bootstrap_num']].apply(lambda x, j: x[x.bootstrap_num==j], boot)
        plot_idx = plot_data.index.get_level_values(x)
        plot_vals = plot_data[y].values
        try:
            popt, _ = sp.optimize.curve_fit(sfp.utils.log_norm_pdf, plot_idx, plot_vals)
        except RuntimeError:
            warnings.warn("For eccentricity %s and frequency space %s, bootstrap %d was not well fit by a log Gaussian and so is skipped" % (data['Eccentricity (degrees)'].unique()[0], data['Frequency'].unique()[0], boot))
        else:
            lines.append(sfp.utils.log_norm_pdf(plot_idx, *popt))
    lines = np.array(lines)
    lines_mean = lines.mean(0)
    cis = np.percentile(lines, ci_vals, 0)
    sns.plt.fill_between(plot_idx, cis[0], cis[1], facecolor=color, alpha=.2, **kwargs)
    sns.plt.plot(plot_idx, lines_mean, color=color, **kwargs)
    return lines

In [ ]:
def compare_hypotheses(df, axis_imgs=None, **kwargs):
    """
    axis_imgs should be a 2d list / array containing 2-tuples. each tuple should contain the relative position of each
        image and the corresponding image to put on an axis. the first dimension corresponds to the first axis, the second
        to the second axis
    """
    tmp_df = df[df.stimulus_superclass=='circular']
#    tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
#    tmp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num', 'hemi'], var_name='Frequency')
    tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num',]]
    tmp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num',], var_name='Frequency')
    tmp_df = tmp_df.rename(columns={'eccen': 'Eccentricity (degrees)'})

    g = sns.FacetGrid(tmp_df, hue='Eccentricity (degrees)', row='Frequency',  palette='Reds', size=5, aspect=2, sharex=False,
                      row_order=['Local spatial frequency (cpd)', 'freq_space_distance'])
    #g.map_dataframe(sfp.utils.plot_mean, 'value', 'amplitude_estimate')
    #g.map_dataframe(sfp.utils.scatter_ci_dist, 'value', 'amplitude_estimate', [50, 50])
    g.map_dataframe(fit_log_norm_ci, 'value', 'amplitude_estimate', ci_vals=[16, 84])
    g.map(sns.regplot, 'value', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False, ci=0)
    sns.plt.subplots_adjust(hspace=.6)
    for i, ax in enumerate(g.axes.flatten()):
        if i==1:
            ax.set_title('Response as function of stimulus')
        elif i==0:
            ax.set_xlim([2**-3.5, 2**4])
            ax.set_title('Response as function of local spatial frequency (cycles / degree)')
        ax.set_xscale('log', basex=2)
        for pos, img in axis_imgs[i]:
            sfp.utils.add_img_to_xaxis(g.fig, ax, img, pos, vmin=0, vmax=255, size=.15)
        ax.xaxis.set_visible(False)
        ax.set_ylabel("Response amplitude estimate")
    #g.ax.set_xlim([2**-3, 2**3.5])
    g.add_legend()
    return g

In [ ]:
with sns.plotting_context('poster'), sns.axes_style('white'):
    g = compare_hypotheses(df, [zip([.025, .6], stim_windows), zip([.05, .6], stims)])
    g.savefig('SF-wl_subj045-results.svg')

Now we create a super summary plot, just showing the peak SF at each eccentricity, which we then overlay on top of the predictions

In [ ]:
tmper_df = tmp_df[['Local spatial frequency (cpd)', 'amplitude_estimate', 'eccen']]

def fit_lnorm(data):
    popt, pcov = sp.optimize.curve_fit(sfp.utils.log_norm_pdf, data.index, data.values)
    return sfp.utils.log_norm_pdf(data.index, *popt)


peak = []
for n, g in tmper_df.groupby('eccen'):
    mn_data = g.groupby(['Local spatial frequency (cpd)'])['amplitude_estimate'].mean()
    peak.append([n, mn_data.index[np.argmax(fit_lnorm(mn_data).values)]])

peak = np.array([((int(i[0])+int(i[2]))/2., j) for i,j in peak])
print peak

In [ ]:
ecc = np.linspace(.01, 9, 50)
RF_scale_factor = 4.
V1_RF_size = np.concatenate([np.ones(len(ecc[ecc<.5]))/RF_scale_factor, np.linspace(1/RF_scale_factor, 4/RF_scale_factor, len(ecc[ecc>=.5]))])
#V2_RF_size = np.concatenate([2*np.ones(len(ecc[ecc<4])), np.linspace(2, 2.5, len(ecc[ecc>=4]))])

Olsson_peak = [2.75, 2.11, 1.76, 1.47,1.24, 1.06, .88, .77, .66, .60]
Olsson_ecc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

with sns.plotting_context('poster', font_scale=1), sns.axes_style('white'):
    # because this doesn't represent data, just intuition, we use this.
#    with plt.xkcd():
    x = np.linspace(.01, 9, 50)
    y = []
    fig, axes = sns.plt.subplots(1,1, figsize=(8,9))
    ax = axes
    # this gives intuitive plots, currently we want the possible hypotheses instead
#         for i in range(3):
#             y.append(10/(x+2)+i)
#             ax.plot(x, y[-1],  label='V%s'%(i+1), color=['r','g','b'][i])
    ax.plot(ecc, 1/V1_RF_size, '-', label='scaling')
    ax.plot(ecc, np.ones(len(ecc))*RF_scale_factor, '--', label='constant')
#        ax.set_ylim((0,8))
    sns.plt.scatter(peak[:, 0], peak[:, 1], c=sns.color_palette('Reds', 6), s=75,)# label='This study')
#    sns.plt.scatter(Olsson_ecc, Olsson_peak, s=75, c= sns.color_palette('Blues', 10), label='Olsson pilot')
    ax.set_xlabel("Receptive field center (degrees)")
    ax.set_ylabel("Peak spatial frequency (cpd)")
    ax.set_title("")
    ax.set_ylim((.5, 4.2))
    ax.set_xlim((0, 11))

    sns.plt.legend(title="hypotheses", loc='best')
    ax.figure.savefig('wl_subj045-results-hypotheses2.svg', bbox_inches='tight')

# Double-check design matrix

In order to use this, run everything except for the `sfp.experiment.run` statement first, which will create pictures, one from each class, in the order the `design_df` thinks they are presented. Then run the `sfp.experiment.run` block on a two-monitor setup so one monitor can display the experiment while you have this notebook open in the other. This will allow you to make sure the stimuli are being ordered correctly.

You can also compare `design_df.index.values` to a picture of the design matrix to make sure things got transferred correctly there as well.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import pyPyrTools.JBhelpers as jbh

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
behav_results = h5py.File('../data/raw_behavioral/2017-Aug-23_Noah_sess1.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
stim_df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

# Array full of the actual stimuli
stim = np.load('../data/stimuli/unshuffled.npy')

# for this, we just want any run, since they all contain the same classes and we don't care about their order
design_df, stim_length, TR = sfp.first_level_analysis.create_design_df(behav_results, stim_df, 1, drop_blanks=False)
design_df = design_df.reset_index(drop=True).set_index("class_idx")

stim_df['class_idx'] = np.floor(stim_df['index']/8)
stim_df = stim_df.set_index('class_idx')
stim_df['Onset time (TR)'] = design_df['Onset time (TR)']
stim_df = stim_df.reset_index().sort('Onset time (TR)')

stim_df = stim_df.drop_duplicates('class_idx')
stim_idx = stim_df.index.values

In [ ]:
data = sfp.experiment.run('../data/stimuli/unshuffled.npy', '../data/stimuli/Noah_run01_idx.npy', None, screen=0)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[:10]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[10:20]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[20:30]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[30:40]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[40:50]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[50:60]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[60:]], ncols=5, zoom=.2)